
# El proceso de clasificación no lineal

No todos los problemas de clasificación se pueden solventar usando clasificadores lineales,
vamos a ver algunas técnicas existentes que permiten sobrepasar las capacidades de los clasificadores
de la sesión anterior.

1. SVM para problemas no lineales.
2. K-Folding y búsqueda de hiperparámetros.
3. Árboles y bosques de decisión.
4. K-NN

## 1. SVM para problemas no lineales.

Una de las razones por las que las SVM gozan de gran popularidad entre los profesionales del aprendizaje automático es que se pueden adaptar fácilmente para resolver problemas de clasificación no lineal.

Para resolver un problema no separable linealmente usando una SVM, transformamos los datos de entrenamiento en un
espacio de características de mayor dimensión a través de una función de mapeo y entrenamos un modelo SVM lineal 
para clasificar los datos en este nuevo espacio de características. Luego, podemos usar la misma función de mapeo 
para transformar datos nuevos para clasificarlos usando el modelo SVM lineal que hemos entrenado.

Un problema que presenta este enfoque de mapeo es que la construcción de las nuevas características es muy
costosa computacionalmente, especialmente si estamos tratando con datos con una alta dimensionalidad. Aquí es 
donde entra en juego el _kernel_ , una función que permite evitar el mapeo a mayores dimensiones.

Se pueden aplicar diferentes kernels, los más usados son aquellos polinómicos y el conocido como RBF ( _Radial
Basis Function_ ).

#### RBF

Es el _Kernel_ más usado, para usarlo necesitamos ajustar dos hiperparámetros: el ya conocido parámetro de
regularización `C` y un segundo parámetro _gamma_ , a mayor valor de este parámetro conduce a un límite de decisión
más suave.

Veamos como influye la modificación de estos parámetros a la decisión del clasificador. **Ver:** Notebook M4_RBF_Kernel

Para usar SVM no lineales en scikit con _kernel_ `RBF`, lo haremos de la siguiente manera:

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf', random_state=0, gamma=0.10, C=1.0)
...

#### Polinómico

Además de los dos parámetros anteriores, hay que determinar el grado del polinomio. A efectos prácticos no es muy
recomendable usar polinomios con grado mayor a 3, ya que corremos el riesgo de sobreajustar el clasificador a los
datos y además el tiempo de entrenamiento crece en gran medida.

Para usar SVM no lineales en scikit con _kernel_ polinómico, lo haremos de la siguiente manera:

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='poly', random_state=0, gamma=0.10, C=1.0)
...

## 2. K-Folding y búsqueda de hiperparámetros.

### Método _holdout_

Este método consiste en separar el conjunto de datos en tres subconjuntos distintos: entrenamiento, validación y _test_.
El conjunto de entrenamiento se usa como viene siendo habitual para entrenar los diferentes modelos. El conjunto de validación se usa
para seleccionar el mejor de los modelos. El conjunto de _test_, que no usamos en ningún caso durante el proceso de entrenamiento, nos
servirá para obtener una idea poco sesgada de la capacidad del modelo de adaptarse a nuevas muestras, sobre este conjunto
de datos será sobre el que obtendremos las métricas finales del modelo.

El proceso de aplicación de esta técnica puede verse en el siguiente gráfico:

![](img/holdout.png)

Este método aunque sencillo de emplear tiene una desventaja, ya que el rendimiento del modelo depende de cómo hemos hecho la
partición de los datos.

### Método _K-Fold_

Esta técnica es más robusta que la anterior, ya que repetimos el método anterior _k_ veces en _k_ subconjuntos
del conjunto de entrenamiento, por tanto obtenemos _k_ modelos y el mismo número de medidas de rendimiento. El resultado
final se obtiene con la media de cada una de las repeticiones realizadas, de esta forma los resultados obtenidos dependen
en menor medida de las particiones que realizamos.

Esta técnica normalmente se usa para obtener los mejores parámetros del modelo a aplicar, es decir encontrar esos
parámetros que maximizan el rendimiento de la métrica que queremos utilizar. Una vez que tenemos los mejores
parámetros, reentrenamos el modelo usando el conjunto de entrenamiento completo y obtenemos las métricas con el
conjunto de _test_.

El proceso de aplicación de esta técnica puede verse en el siguiente gráfico:

<img src="img/Kfold.png" alt="kfold" width="600"/>

La pregunta que podemos hacernos es: ¿Cómo seleccionar este parámetro _k_ de forma correcta?

Por último, existe una variante de esta técnica llamada _stratified k-fold_ en el que las proporciones entre clases
se mantienen en cada una de las iteraciones, esto es importante cuando tenemos problemas desbalanceados.


### Ajustamos el modelo usando una búsqueda exhaustiva ( _Grid Search_ )

Los parámetros de nuestro modelo que podemos ajustar manualmente, es decir que no son aprendidos de los datos de entrenamiento,
se llaman hiper-parámetros. Ajustar su valor de forma intuitiva o mediante sucesiones de pruebas y errores puede ser
una tarea larga, por no decir imposible en el caso de modelos con muchos parámetros como pueden ser los _Random Forests_.
Existe una técnica de búsqueda exhaustiva de los valores óptimos de los hiperparámetros conocida con el nombre de _Grid Search_ que nos permite automatizar esta búsqueda
penalizando el coste temporal del proceso de entrenamiento.

### Combinación de _K-Fold_ con _Grid Search_

La combinación de las dos técnicas explicadas anteriormente es una de las más utilizadas, se conoce con el nombre de
_nested cross-validation_. En este caso tenemos dos bucles anidados: el más externo en el que se divide el conjunto
de entrenamiento usando K-folding y otro interno en el que se realiza la búsqueda de los mejores hiper-parámetros.

El esquema que se sigue es el siguiente:

<img src="img/grid_search_k_fold.png" width="600"/>

### Ajuste y Evaluación del modelo

En sesiones anteriores habíamos descrito el flujo del proceso de aprendizaje, los últimos pasos que formaban parte del
proceso eran el ajuste y la evaluación del modelo. En esta práctica aplicaremos lo explicado hasta ahora usando las herramientas que
_Scikit_ pone a nuestro alcance.

Los pasos a seguir son:

1. Separación del conjunto de datos: entrenamiento y test.
2. Definición de los parámetros a ajustar. El formato de la parrilla es un diccionario donde la clave es el nombre del parámetro y
el valor una lista con todos los valores a probar.
3. Aplicamos la búsqueda exhaustiva (_grid search_) junto con _k-folding_. Usaremos la función _GridSearchCV_ [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html?highlight=gridsearchcv#sklearn.model_selection.GridSearchCV): `GridSearchCV(estimator, param cv=None, verbose=0)`.
4. Entrenamos con el mejor modelo obtenido y el conjunto de entrenamiento entero. Podemos obtener el mejor modelo resultante
del paso anterior con el atributo `best_estimator_` del objeto `GridSearchCV`.
5. Obtener los resultados finales con el conjunto de _test_.


## 3. Árboles y bosques de decisión.

Podemos pensar en un árbol de decisión como un modelo como que organiza nuestros datos al
tomar decisiones basadas en hacer una serie de preguntas binarias.

Según las características de nuestro conjunto de entrenamiento, el árbol de decisión aprende una serie de
decisiones binarias para acabar infiriendo las etiquetas de clase de las muestras de dicho conjunto.

Usando el algoritmo de decisión, comenzamos en la raíz del árbol y dividimos los datos en la característica que da
como resultado la mayor ganancia de información (IG). La construcción del árbol es un proceso iterativo, podemos
repetir este procedimiento de división en cada nodo hijo hasta que las hojas del árbol sean puras, es decit hasta que
todas las muestras de cada nodo pertenezcan a la misma clase. En la práctica, esto puede resultar
en un árbol muy profundo con muchos nodos, lo que puede conducir fácilmente a un sobreajuste (_overfitting_). Por lo
tanto, normalmente estableceremos un límite para la profundidad máxima del árbol.

### Maximizar la obtención de información

Para dividir los nodos usando las características más informativas (las mejores), necesitamos definir la función que
queremos optimizar a través del algoritmo de aprendizaje. Nuestra función objetivo consiste en maximizar la ganancia
de información en cada una de las sucesivas divisiones, definida a continuación:

$$ IG(D_p, f) = I(D_p) - \sum_{j=1}^m \frac{N_j}{N_p}I(D_j)$$

Los árboles que encontramos en la librería _scikit_ son árboles binarios, entonces podemos reformular la función de la
siguiente manera:

$$ IG(D_p, f) = I(D_p) - \frac{N_{izquierda}}{N_p}I(D_{izquierda}) - \frac{N_{derecha}}{N_p}I(D_{derecha}) $$

En este contexto $I$ es la medida de impureza de un nodo. Podemos usar dos diferentes, la entropía o el índice _Gini_,
aunque miden aspectos diferentes a efectos prácticos obtendremos resultados muy similares usando cualquiera de los dos.

En scikit podemos usarlos de la siguiente manera:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)
tree.fit(X_train, y_train)
...

### Bosques aleatorios de decisión

Los bosques aleatorios (conocidos como _Random Forest_ ) han ganado una gran popularidad en las aplicaciones de
aprendizaje automático durante la última década debido a su buen rendimiento de clasificación, escalabilidad y 
facilidad de uso. Intuitivamente, un bosque aleatorio puede considerarse como un conjunto de árboles de decisión. 
La idea detrás del aprendizaje en conjunto es combinar clasificadores "débiles" para, entre todos juntos, 
construir un modelo más robusto, que tenga un mejor error de generalización y sea menos susceptible al 
sobreajuste. Nuestro objetivo final, es reducir la varianza de un árbol de decisión. 

El algoritmo se puede resumir cuatro pasos:

   1. Seleccionar una muestra ( _bagging_ ) aleatoria de tamaño $n$ (se elige de forma aleatoria $n$ muestras del
conjunto de entrenamiento con reemplazo).
   2. Construir un árbol de decision a partir de la muestra seleccionada en el paso anterior. En cada nodo del árbol:
        1. Seleccionar aleatoriamente $d$ características (sin reemplazo).
        2. Dividir el nodo utilizando la característica que proporcione la mejor división de acuerdo con la función
        objetivo.
   3. Repetir los pasos 1 y 2 _k_ veces.
   4. Para hacer una predicción se debe agregar la predicción por cada árbol de forma individual y se asignará aquella
   etiqueta de clase que cuente con la mayoría de votos.

Aunque los bosques aleatorios no ofrecen el mismo nivel de interpretabilidad que los árboles de decisión, una gran
ventaja de los bosques aleatorios es que no tenemos que preocuparnos tanto por elegir buenos valores de hiperparámetros.
Por lo general, no necesitamos podar el bosque aleatorio, ya que el modelo es bastante robusto al ruido
de los árboles de decisión individuales. El único parámetro que realmente debemos preocuparnos en la práctica es el
número de árboles _k_ que elegimos para este modelo. Por lo general, cuanto mayor sea el número de
árboles, mejor será el rendimiento del clasificador a expensas de un mayor costo computacional.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(criterion='entropy', n_estimators=10,random_state=1)

### Boosting

_Boosting_ es otra técnica para crear una colección de predictores. En esta técnica, los modelos se entrenan secuencialmente: los primeros ajustan modelos simples a los datos y luego analizan esos datos en busca de errores de clasificación. En otras palabras, ajustamos árboles consecutivos (muestra aleatoria) y en cada paso, el objetivo es resolver el error neto del árbol anterior. 

Un ejemplo de este tipo de aprendizaje es [Gradient Boosting](https://scikit-learn.org/stable/modules/ensemble.html#gradient-tree-boosting)

<img src="img/boosting.png" alt="kfold" width="600"/>

Ventajas de usar la técnica Gradient Boosting:
- Admite diferentes funciones de pérdida.
- Funciona bien con las características altamente relacionadas.

Desventajas de usar la técnica Gradient Boosting:
- Propenso al sobreajuste.
- Requiere un ajuste cuidadoso de diferentes hiperparámetros 




## 4. K-nearest neighbors

K-nearest neighbors (KNN) es fundamentalmente diferente de los algoritmos de aprendizaje que hemos visto hasta el
momento. Este ess un ejemplo típico de aprendizaje basado en ejemplos, ya que no aprende una función discriminativa de
los datos de entrenamiento, sino que memoriza el conjunto de datos de entrenamiento en su lugar.

El algoritmo KNN en sí es bastante sencillo y se puede resumir en los siguientes pasos:

1. Se elige un número de _clústers_ $k$ y una métrica de distancia.
2. Se buscan los $k$ vecinos más cercanos de la muestra que queremos clasificar.
3. Se asigna la etiqueta de la clase con la mayoría de votos.

La principal ventaja de este enfoque basado en memorizar datos es que el clasificador se adapta inmediatamente a medida
que recopilamos nuevos datos de entrenamiento. Sin embargo, la desventaja es que la complejidad computacional para clasificar
nuevas muestras crece linealmente con la cantidad de muestras en el conjunto de datos de entrenamiento en el peor de los
casos.

Por una parte debemos asegurarnos de elegir una métrica de distancia que sea apropiada para las características del
conjunto de datos. A menudo, la medida de distancia euclidiana se usa para muestras con valores reales, por ejemplo.
Sin embargo, si usamos una medida de distancia euclidiana, también es importante estandarizar los datos para que cada
característica contribuya por igual a la distancia. También podemos usar la distancia manhattan si usamos datos
categóricos.

Su uso en scikit es el siguiente:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# En este caso la métrica minkowski con p=2 es igual a usar la distancia euclídea.
# minkowski con p=1 es lo mismo que usar la distancia manhattan
knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
knn.fit(X_train, y_train)
knn.predict(X_test)
